<a href="https://colab.research.google.com/github/sulova/Project-Colab/blob/main/notebooks/Canada_Errosion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the earthengine-api.
!pip install -q earthengine-api


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=C8vOyWeczRsgOWIMW1aNO31H8FlMpJVMW80nxIjixK4&tc=y6Wu9KCnLq5O0HwM1rkecbBmktTMLNy9EAc9NKjX-jI&cc=1B7vOXag2tc-5za3S7sK0Tc4aPTqDO4wvySE_OMHin8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh713os3yxut0MDt42cKrnbjfNVq354zoWowkhB7MjiQ_qMoJF4ua4s

Successfully saved authorization token.


In [7]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QQVr4OUUrNzlPXTg44jto_IazMhPseMZxB7Y-b-doy8&tc=sZXoysekantQxFwfaMtUyCmslIiMYr4GXqLBIk2ZpWU&cc=3iDvLFZTEZQM9C3DE8U7aGw03EiDWglATByHEonUo8A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7QUYabqwqth2hJCK83F-Qo6ovtH4_aT8DaIHWNNUCfa9R8cr1XoiE

Successfully saved authorization token.


In [8]:
# install geemap library
%%capture
!pip install geemap

In [9]:
# import geemap library
import geemap

In [12]:
# Define AOI
AOI = ee.Geometry.Polygon([[[
              -78.59962463378903,
              51.535391545426165
            ],
            [
              -78.59962463378903,
              51.44304121677976
            ],
            [
              -78.8605499267578,
              51.44304121677976
            ],
            [
              -78.8605499267578,
              51.535391545426165
            ],
            [
              -78.59962463378903,
              51.535391545426165
            ]]])


In [13]:
## Let's set some dates for testing - January 2022.
startDate = '2015-01-01'
endDate = '2022-12-01'

## First we'll get some Sentinel-2 data
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filter([
           ee.Filter.date(startDate, endDate),
           ee.Filter.bounds(AOI)
           ])

## Finally, make a S2 composite and clip to our AOI. Easy!
s2_median = s2.median().clip(AOI)

## Let's visualise it.
Map = geemap.Map( height=400, draw = True)
Map.add_basemap("TERRAIN")
Map.centerObject(AOI, 8)
Map.addLayer(AOI, {}, 'AOI')
Map.addLayer(s2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3500},'S2 median ')
Map

Map(center=[51.489273161524665, -78.7300872802737], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
## Get the s2 cloudless image collection and apply data and bounds filter.
s2_cloudless = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filter([
           ee.Filter.date(startDate, endDate),
           ee.Filter.bounds(AOI)
           ])


join = ee.Join.inner().apply(**{
        'primary': s2,
        'secondary': s2_cloudless,
        'condition': ee.Filter.equals(**{'leftField': 'system:index', 
                                         'rightField': 'system:index'})
        })

def merge_collections (feature):
  return ee.Image(feature.get('primary')).addBands(feature.get('secondary'))

s2_cloudless_merged = ee.ImageCollection(join.map(merge_collections))

print(s2_cloudless_merged.first().bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60', 'probability']


In [20]:
probability_threshold = 20

s2_cloud_masked = s2_cloudless_merged.map(
    lambda image: image.updateMask(image.select('probability').lt(probability_threshold))) 

## Let's visualize the new composite.
s2_cloud_masked_median = s2_cloud_masked.median().clip(AOI)

## Let's visualise it.
Map = geemap.Map( height=400, draw = True)
Map.add_basemap("TERRAIN")
Map.centerObject(AOI, 13)
Map.addLayer(AOI, {}, 'AOI')
left_layer = geemap.ee_tile_layer(s2_median, {'bands': ['B4', 'B3', 'B2'], 
                                              'min': 0, 'max': 2000},'S2 median ')
right_layer = geemap.ee_tile_layer(s2_cloud_masked_median, 
                                   {'bands': ['B4', 'B3', 'B2'], 
                                    'min': 0, 'max': 2500},'S2 Cloud Masked Median ')
Map.split_map(left_layer, right_layer)
Map

Map(center=[51.489273161524665, -78.7300872802737], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [22]:
def add_s2_water_probability (image):
  ## Create the required input bands.
  ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
  ndwi = image.normalizedDifference(['B3','B8']).rename('ndwi')

  ## Add them and compute the water probability.
  water_probability = image.addBands([ndvi, ndwi]).expression(
      "1 / (1 + exp(- (0.845 + (2.14 * b('ndvi')) + (13.5 * b('ndwi')))))"
      ).multiply(100).rename(['water_probability'])

  ## return the original image + the water probability.
  return image.addBands([water_probability, ndvi, ndwi])

## We'll map that function over our cloud masked collection
s2_water_collection = s2_cloud_masked.map(add_s2_water_probability)


In [23]:
s2_water_collection.size()

In [53]:
months = ee.List.sequence(1, 12)
years = ee.List.sequence(2021,2022,1)

def function1(y):
    def function2(m):
        return s2_water_collection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).mean().set('month', m).set('year', y)
    return months.map(function2)


s2_water_collection_month= ee.ImageCollection.fromImages(years.map(function1).flatten())
s2_water_collection_month


In [54]:
# Define the function of adding the time band.
def add_time_band (image):

  # Returns a Date given year, month, day (2017-01-01).
  basedate = ee.Date.fromYMD(2021, 1, 1) 

  # Extract the date from the properties within each image.
  ymd = ee.Date.fromYMD(image.getNumber('year'), image.getNumber('month'), 1)

  # compute the difference between the dates, with units month, and cast as an image.
  time_difference = ee.Image(ymd.difference(basedate, 'month')).float().rename('time')

  # Add a new time band formatted as YYYY-MM-dd.
  return image.addBands(time_difference).set(
      'ymd', ymd.format('YYYY-MM-dd'),
      'system:time_start', ymd.millis())
  
s2_water_collection_time = s2_water_collection_month.map(add_time_band)


In [60]:

linear_fit = s2_water_collection_time.select(['time', 'water_probability']).reduce(ee.Reducer.linearFit(), 2).clip(AOI)
  
# Visualize.
Map = geemap.Map(height=700, draw = True)
Map.add_basemap("TERRAIN")
Map.centerObject(AOI, 12)
red_blue = ['#ca0020','#FFFFFF00','#0571b0']
Map.addLayer(linear_fit, {'bands': ['scale'], 'min': -10, 'max': 10, 'palette': red_blue},'scale')
Map.addLayer(s2_cloud_masked_median,{'bands': ['B4', 'B3', 'B2'], 
                                    'min': 0, 'max': 2500},'S2 Cloud Masked Median ')

Map.add_colorbar({'bands': ['scale'], 'min': 4, 'max': 1, 'palette': red_blue}, label="[°C]", layer_name="Warmer (scale)")
Map


The provided color (#FFFFFF00) is invalid. Using the default black color.
'##FFFFFF00' is not in web format. Need 3 or 6 hex digit.


Map(center=[51.489273161524665, -78.7300872802737], controls=(WidgetControl(options=['position', 'transparent_…

In [43]:
# Probability must be increasing (getting wetter).
scale_mask = linear_fit.select('scale').gte(1)

# Offset must have been land in the begginning.
offset_mask = linear_fit.select('offset').lte(0) 

# Combine the masks.
erosion = scale_mask.And(offset_mask)

# Keep only these areas in our visualisation. 
linear_fit_masked = linear_fit.updateMask(erosion)

# Visualise.
Map = geemap.Map( zoom=10, height= 700, draw = True)
Map.centerObject(AOI, 11)
Map.addLayer(erosion, {'min': 0, 'max': 1, 'palette': red_blue}, 'erosion mask', False)
Map.addLayer(linear_fit_masked, {'bands': ['offset'], 'min': 0, 'max': 100, 'palette': red_blue},'Erosion (offset)')
Map.addLayer(linear_fit_masked, {'bands': ['scale'], 'min': 0, 'max': 3, 'palette': red_blue},'Erosion (scale)')
Map

Map(center=[51.489273161524665, -78.7300872802737], controls=(WidgetControl(options=['position', 'transparent_…